# Code to get filtered points

In [1]:
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import transform_matrix
#from nuscenes.utils.data_classes import RadarPointCloud
from my_radar import RadarPointCloud
from pyquaternion import Quaternion
import numpy as np
import cv2
import os.path as osp
import math


#RadarPointCloud.disable_filters()  #to make the radar point denser
USED_SENSOR = {'RADAR_FRONT','RADAR_FRONT_LEFT','RADAR_FRONT_RIGHT','RADAR_BACK_LEFT','RADAR_BACK_RIGHT'}
accum_radar_points = np.empty((3,1))

nusc = NuScenes(version='v1.0-trainval', dataroot='E:\data/trainval', verbose=True)
for one_instance in nusc.instance:
    cata = nusc.get('category', one_instance['category_token'])
    if cata['name'] == 'vehicle.car':
        ann_tokens = nusc.field2token('sample_annotation', 'instance_token', one_instance['token'])
        ann_tokens = set(ann_tokens)
        for one_token in ann_tokens:
            one_ann = nusc.get('sample_annotation', one_token)
            global_from_anno = transform_matrix(one_ann['translation'],Quaternion(one_ann['rotation']),inverse=False)
            global_from_anno = global_from_anno[:2,:2].T
            sample = nusc.get('sample', one_ann['sample_token'])
            w =  one_ann['size'][0]
            l =  one_ann['size'][1]
            bound_l = l * 0.6
            bound_w = w * 0.6

            for radar in USED_SENSOR:        
                sample_data = nusc.get('sample_data', sample['data'][radar])
                filename = osp.join(nusc.dataroot, sample_data['filename'])
                radar_pcs = RadarPointCloud.from_file(file_name=filename,invalid_states=[0], 
                                        dynprop_states=[6],ambig_states=[3],pdh0=[1]) 
                                        #only "cross-moving, unambiguious doppler, valid, artifect less than 25%" is reserved
                ego_pose = nusc.get('ego_pose', sample_data['ego_pose_token'])   
                sensor_pose = nusc.get('calibrated_sensor',sample_data['calibrated_sensor_token']) #ego-car coordinate
                car_from_sensor = transform_matrix(sensor_pose['translation'], Quaternion(sensor_pose['rotation']), inverse=False)
                global_from_car = transform_matrix(ego_pose['translation'],Quaternion(ego_pose['rotation']), inverse=False)
                trans_matrix = np.dot(global_from_car,car_from_sensor)
                inv_trans_matrix = np.linalg.inv(trans_matrix)
                
                
                ann_gc = np.hstack((one_ann['translation'],np.array([1]))).reshape(4,1)
                ann_sc = np.dot(inv_trans_matrix, ann_gc)
                yaw_sc = math.atan2(ann_sc[1],ann_sc[0])  #range[-pi,pi]
                orientation_sc = np.dot(inv_trans_matrix[:2,:2], global_from_anno.T)
                #orientation_sc = global_from_anno
                #phi_sc = math.acos(orientation_sc[0,0])   #range[0,pi]
                #if orientation_sc[0,1] > 0:
                #    phi_sc = phi_sc - math.pi
                phi_sc = math.atan2(orientation_sc[1,0],orientation_sc[0,0])
                xi_sc =  yaw_sc - phi_sc


                radar_pcs.transform(trans_matrix)   #transform radar points from sensor-coordinate to global coordinate
                radar_points = radar_pcs.points[:2,:]
                anno_pose = np.array(one_ann['translation'])[:2,].reshape(2,1)
                radar_points_ac = radar_points - anno_pose                       #annotation coordinate
                radar_points_uc = np.dot(global_from_anno,radar_points_ac)       #unit coordinate                   
                for i in range(0,radar_points_uc.shape[1]):
                    if -bound_l < radar_points_uc[0,i] < bound_l and  -bound_w < radar_points_uc[1,i] < bound_w:      
                        uc = np.array([2*radar_points_uc[0,i]/l, 2*radar_points_uc[1,i]/w, xi_sc]).reshape(3,1)         
                        accum_radar_points = np.hstack((accum_radar_points,uc))
np.savetxt('test_trainval_01192245.txt',accum_radar_points,fmt='%.2f')   

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 69.589 seconds.
Reverse indexing ...
Done reverse indexing in 12.0 seconds.
